In [1]:
#Funcion para lectura de datos .gpx
import gpxpy
import pandas as pd

def read_gpx(file: str) -> pd.DataFrame:
    df = None
    points = []
    with open(file) as f:
        gpx = gpxpy.parse(f)

    for segment in gpx.tracks[0].segments:
        for p in segment.points:
            points.append({
                'time':p.time,
                'latitude':p.latitude,
                'longitude':p.longitude,
                'elevation':p.elevation
            })
    df = pd.DataFrame.from_records(points)
    return df

In [2]:
#Datos
df1 = read_gpx('recovery.01-Mar-2022-1533.gpx')
df2 = read_gpx('recovery.05-Mar-2022.1025.gpx')
df3 = read_gpx('recovery.25-May-2022-0907.gpx')

In [3]:
df1

,time,latitude,longitude,elevation
0,2022-03-01 20:33:48+00:00,6.297476,-75.578192,1668.879307
1,2022-03-01 20:33:49+00:00,6.297476,-75.578146,1668.386143
2,2022-03-01 20:33:57+00:00,6.297460,-75.578169,1667.904453
